In [9]:
import numpy as np
import yfinance as yf
tickers=['AMZN','GOOG','TSLA']
ohlcv_data={}
for ticker in tickers:
    temp=yf.download(ticker,period='1mo',interval='5m')
    temp.dropna(how='any',inplace=True)
    ohlcv_data[ticker]=temp

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [2]:
def ATR(DF,n=14):
    df=DF.copy()
    df['H-L']=df['High']-df['Low']
    df['H-PC']=df['High']-df['Adj Close'].shift(1)
    df['L-PC']=df['Low']-df['Adj Close'].shift(1)
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR']=df['TR'].ewm(com=n,min_periods=n).mean()
    return df['ATR']

In [3]:
for ticker in ohlcv_data:
    ohlcv_data[ticker]['ATR']=ATR(ohlcv_data[ticker])

In [4]:
ohlcv_data['AMZN']

,Open,High,Low,Close,Adj Close,Volume,ATR
Datetime,,,,,,,
2023-08-07 09:30:00-04:00,140.990005,141.240005,139.910004,140.079193,140.079193,4420439,NaN
2023-08-07 09:35:00-04:00,140.059998,140.729996,139.660004,140.419998,140.419998,1744133,NaN
2023-08-07 09:40:00-04:00,140.429993,140.509995,139.910004,139.970001,139.970001,2715302,NaN
2023-08-07 09:45:00-04:00,139.975098,140.139999,139.259995,139.280106,139.280106,1439452,NaN
2023-08-07 09:50:00-04:00,139.520004,140.190002,139.304993,139.914993,139.914993,1484756,NaN
...,...,...,...,...,...,...,...
2023-09-05 15:35:00-04:00,136.940002,137.080002,136.904999,137.050003,137.050003,420497,0.167046
2023-09-05 15:40:00-04:00,137.050003,137.225006,137.039993,137.216293,137.216293,446335,0.168244
2023-09-05 15:45:00-04:00,137.214996,137.250000,137.009995,137.070007,137.070007,514873,0.173028


In [6]:
def ADX(DF, n = 20):
    df = DF.copy()
    df['ATR'] = ATR(df, n)
    df['upmove'] = df['High'] - df['High'].shift(1)
    df['downmove'] = df['Low'].shift(1) - df['Low']
    df['+dm'] = np.where((df['upmove']>df['downmove']) & (df['upmove']>0),df['upmove'],0)
    df['-dm'] = np.where((df['downmove']>df['upmove']) & (df['downmove']>0),df['downmove'],0)
    df['+di'] = 100*(df['+dm']/df['ATR']).ewm(span = n, min_periods=n).mean()
    df['-di'] = 100*(df['-dm']/df['ATR']).ewm(span = n, min_periods=n).mean()
    df['ADX'] = 100*abs((df['+di'] - df['-di'])/(df['+di'] + df['-di'])).ewm(span = n, min_periods=n).mean()
    return df['ADX']

In [10]:
for ticker in ohlcv_data:
    ohlcv_data[ticker]['ADX'] = ADX(ohlcv_data[ticker],20)

In [12]:
ohlcv_data['AMZN']

,Open,High,Low,Close,Adj Close,Volume,ADX
Datetime,,,,,,,
2023-08-07 09:30:00-04:00,140.990005,141.240005,139.910004,140.079193,140.079193,4420439,NaN
2023-08-07 09:35:00-04:00,140.059998,140.729996,139.660004,140.419998,140.419998,1744133,NaN
2023-08-07 09:40:00-04:00,140.429993,140.509995,139.910004,139.970001,139.970001,2715302,NaN
2023-08-07 09:45:00-04:00,139.975098,140.139999,139.259995,139.280106,139.280106,1439452,NaN
2023-08-07 09:50:00-04:00,139.520004,140.190002,139.304993,139.914993,139.914993,1484756,NaN
...,...,...,...,...,...,...,...
2023-09-05 15:35:00-04:00,136.940002,137.080002,136.904999,137.050003,137.050003,420497,24.455567
2023-09-05 15:40:00-04:00,137.050003,137.225006,137.039993,137.216293,137.216293,446335,24.901864
2023-09-05 15:45:00-04:00,137.214996,137.250000,137.009995,137.070007,137.070007,514873,24.765260
